In [1]:
import time
import re
import spacy
import os

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
nlps = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
config = {"punct_chars": None}
nlps.add_pipe("sentencizer", config=config)

In [4]:
number_regex = "\\b[0-9]+[0-9\.,\-]*[gcjifhbe]{0,1}\\b"
def getValues(paragraph):
    out = {}
    for num_match in re.finditer(number_regex, paragraph):
        num_beg = num_match.span()[0]
        num_end = num_match.span()[1]
        out[num_beg] = num_end
    return out

In [5]:
def getCatSpans(paragraph):
    out = {}
    prev_beg = 0
    cat_end= 0
    prev_label = "OTHER"
    cat_beg_end_map = {}
    cat_beg_label_map = {}
    for cat_match in re.finditer("(DRINKING.WATER|GROUND.WATER|SURFACE.WATER|DRINKINGWATER|SURFACEWATER|GROUNDWATER)".lower(), paragraph.lower()):
        cat_beg = cat_match.span()[0]
        cat_end = cat_match.span()[1]
        cat_beg_end_map[prev_beg] = cat_end
        cat_beg_label_map[prev_beg] = prev_label
        prev_beg = cat_beg
        prev_label = cat_match.group(0)
    cat_beg_end_map[prev_beg] = len(paragraph)
    cat_beg_label_map[prev_beg] = prev_label
    return cat_beg_end_map, cat_beg_label_map

In [6]:
def mergeAtParaLevel(prediction, offset, finalPrediction):
    for entity in prediction:
        entity["start"]+=offset
        entity["end"]+=offset
        finalPrediction.append(entity)
    return finalPrediction

In [7]:
def mapValues(finalPrediction, valuesBegEnd, paragraph, cat_beg_end_map, cat_beg_label_map):
    for entity in finalPrediction:
        min_val = 10000000
        out = -1
        beg = entity["start"]
        for val in valuesBegEnd:
            isContainNewLine = False
            if val < beg:
            #    if '\n' in paragraph[val:beg]:
                    isContainNewLine = True
            if beg < val:
                if '\n' in paragraph[beg:val]:
                    isContainNewLine = True
            if abs(val-beg)<min_val and not isContainNewLine:
                out = val
                min_val = abs(val-beg)
        if out != -1:
          entity["value_start"] = out
          entity["value_end"] = valuesBegEnd[out]
          entity["value"] = paragraph[out:valuesBegEnd[out]]
        for cat_beg in cat_beg_end_map:
            cat_end = cat_beg_end_map[cat_beg]
            if cat_beg<=beg and beg<=cat_end:
                entity["category"] = cat_beg_label_map[cat_beg]
    return finalPrediction

In [8]:
def predict(paragraph):
    finalPrediction = []
    valuesBegEnd = getValues(paragraph)
    cat_beg_end_map, cat_beg_label_map = getCatSpans(paragraph)
    sentences = nlps(paragraph)
    offset = 0
    for sentence in sentences.sents:
        if sentence.text:
            prediction = nlp(sentence.text)
            finalPrediction = mergeAtParaLevel(prediction, offset, finalPrediction)
        offset+=len(sentence.text)
    #print(finalPrediction, valuesBegEnd, paragraph, cat_beg_end_map, cat_beg_label_map)
    finalPrediction = mapValues(finalPrediction, valuesBegEnd, paragraph, cat_beg_end_map, cat_beg_label_map)
    return finalPrediction

In [28]:
from transformers import pipeline
from transformers import AutoModelForTokenClassification, AutoTokenizer, BertForTokenClassification

import torch
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint1/")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint1/")
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
def preprocessing(lines):
  new_line = "";
  preprocessed_input = "";
  for line in lines:
    if(line.strip() != ""):
      new_line = new_line +" "+ line.strip()
      continue;
    if(new_line != ""):
      preprocessed_input = preprocessed_input +"\n"+ new_line
    new_line = ""
  if(new_line != ""):
    preprocessed_input = preprocessed_input +"\n"+ new_line
  return preprocessed_input

In [ ]:
input = "                    MAXIMUM CONTAMINANT LEVELS IN DRINKING WATER\n\nA. Applicability.\n\n  This regulation shall apply to each public water system, unless the water system meets all of the following\nconditions:\n\n  (1) Consists only of distribution and storage facilities (and does not have any collection and treatment\nfacilities);\n\n  (2) Obtains all of its water from, but is not owned or operated by, a public water system to which such\nregulations apply;\n\n  (3) Does not sell water to any person; and\n\n  (4) Is not a carrier which conveys passengers in interstate commerce.\n\nB. Maximum Contaminant Levels for Inorganic Chemicals.\n\n  (1) The Maximum Contaminant Levels (MCLs) for inorganic contaminants specified in R.61-\n58.5(B)(2) shall apply to all public water systems. Compliance with maximum contaminant levels for\ninorganic chemicals are calculated pursuant to Section (C) below:\n\n  (2) The maximum contaminant levels for inorganic chemicals are as follows:\n\n       Contaminant                                                 Level (mg/L)\n\n        (a)     Arsenic                                                    0.010**\n        \n        (b)     Asbestos                                           7 Million Fibers/liter\n                                                                   (longer than 10m)\n        (c)     Barium                                                     2.0\n        \n        (d)     Cadmium                                                     0.005\n        \n        (e)     Chromium                                                    0.1\n        \n        (f)     Fluoride                                                    4.0\n        \n        (g)     Mercury                                                     0.002\n        \n        (h)     Nitrate (as Nitrogen)                                       10\n        \n        (i)     Nitrite (as Nitrogen)                                       1\n        \n        (j)     Total Nitrate and Nitrite                                   10\n                (as Nitrogen)\n        \n        (k)     Selenium                                                   0.05\n        \n        (l)     Antimony                                                   0.006\n        \n        (m)     Beryllium                                                  0.004\n        \n        (n)     Cyanide (as free Cyanide)                                  0.2\n        \n        (o)     Thallium                                                   0.002\n\n** The MCL for arsenic is 0.05 milligrams per liter (mg/L) for all public water systems until January 23,\n2006.\n\n   (3) At the discretion of the Department, nitrate levels not to exceed twenty milligrams per liter may be\nallowed in a non-community water system if the supplier of water demonstrates to the satisfaction of the\nDepartment that:\n\n     (a) Such water will not be available to children under six months of age; and,\n\n     (b) The non-community water system is meeting the public notification requirements under R.61-\n58.6(E)(9), including continuous posting of the fact that nitrate levels exceed ten (10) milligrams per liter\nand the potential health effects of exposure; and,\n\n     (c) No adverse health effects shall result from the consumption of this water.\n\nC. Primary Inorganic Chemical Sampling and Analytical Requirements.\n\n   (1) The monitoring requirements for inorganic contaminants specified in Section B (2)(b), (c), (d), (e),\n(g), (k), (l), (m), (n), and (o) above apply to community water systems and non-transient non- community\nwater systems. The monitoring requirements for inorganic contaminants specified in Section B (2)(a) and\n(f) above only apply to community water systems. Beginning January 22, 2004, the monitoring\nrequirements for the inorganic contaminant specified in Section B (2)(a) above will apply to community\nwater systems and non-transient, non-community water systems. The monitoring required for inorganic\ncontaminants specified in Section B (2)(h), (i) and (j) above apply to community, non-transient non-\ncommunity and transient non-community water systems.\n\n  (2) Analytical methods used to comply with Section B above, shall be made using EPA- approved\nmethods listed in 40 CFR 141. Analyses for the purpose of determining compliance with Section B above\nare required as follows:\n\n     (a) Analyses for all community water systems utilizing surface water sources, in whole or in part,\nshall be completed within one year following the effective date of this regulation. These analyses shall be\nrepeated at yearly intervals.\n\n     (b) Analyses for all community water systems utilizing only groundwater sources shall be completed\nwithin two years following the effective date of this regulation. These analyses shall be repeated at three-\nyear intervals.\n\n     (c) For non-community water systems, whether supplied by surface or groundwater sources, analyses\nfor nitrate shall be completed within two years following the effective date of this regulation. These analyses\nshall be repeated at intervals determined by the Department.\n\n    (d) The Department shall have the authority to determine compliance or to initiate enforcement action\nbased upon analytical results and other information compiled by the Department.\n\n  (3) If the result of an analysis made pursuant to subsection (2) above indicates that the level of any\ncontaminant listed in Section B above exceeds the maximum contaminant level, the supplier of water shall\nreport to the Department within seven days.\n\n   (4) When the maximum contaminant level for any contaminant listed in Section B above is exceeded\nas determined in accordance with subsection (15) below, the supplier of water shall notify the Department\nand give notice to the public pursuant to R. 61-58.6, Reports, Record Retention, and Public Notification,\nSections B and E. Monitoring after public notification shall be at a frequency designated by the Department\nand shall continue until the maximum contaminant level has not been exceeded in two successive samples\nor until a monitoring schedule as a condition to a variance, exemption, or enforcement action shall become\neffective.\n\n  (5) The provisions of subsections (3) and (4) above notwithstanding, compliance with the maximum\ncontaminant level for nitrate and nitrite shall be determined in accordance with subsection (12)(b) below.\n\n   (6) For the initial analyses required by items (2)(a), (b), or (c) above, data for surface waters acquired\nwithin one year prior to the effective date and data for groundwater acquired within three years prior to the\neffective date of this regulation may be substituted at the discretion of the Department. Analyses conducted\nto determine compliance with Section B above shall be made in accordance with the analytical methods\nadopted by the Department.\n\n  (7) Monitoring for the purpose of determining compliance with the maximum contaminant levels\nspecified in Section B (2) above, shall be conducted as follows:\n\n     (a) Groundwater systems shall take a minimum of one sample at every entry point to the distribution\nsystem which is representative of each well after treatment (hereafter called a sampling point) beginning in\nthe initial compliance period. The system shall take each sample at the same sampling point unless\nconditions make another sampling point more representative of each source or treatment plant.\n\n     (b) Surface water systems shall take a minimum of one sample at every entry point to the distribution\nsystem after any application of treatment or in the distribution system at a point which is representative of\neach source after treatment (hereafter called a sampling point) beginning in the initial compliance period.\nThe system shall take each sample at the same sampling point unless conditions make another sampling\npoint more representative of each source or treatment plant. [Note: For purposes of this paragraph, surface\nwater systems include systems with a combination of surface and ground sources.]\n\n      (c) If a system draws water from more than one source and the sources are combined before\ndistribution, the system must sample at an entry point to the distribution system during periods of normal\noperating conditions (i.e., when water is representative of all sources being used).\n\n     (d) The Department may reduce the total number of samples which must be analyzed by allowing the\nuse of compositing. Composite samples from a maximum of five samples are allowed, provided that the\ndetection limit of the method used for analysis is less than one-fifth of the MCL. Compositing of samples\nmust be done in the laboratory.\n\n        (i) If the concentration in the composite sample is greater than or equal to one-fifth of the MCL\nof any inorganic chemical, then a follow-up sample must be taken within 14 days at each sampling point\nincluded in the composite. These samples must be analyzed for the contaminants which exceeded one-fifth\nof the MCL in the composite sample. Detection limits for each analytical method and inorganic contaminant\nshall be in accordance with those listed in 40 CFR 141\n\n         (ii) If the population served by the system is greater than 3,300 persons, then compositing may\nonly be permitted by the Department at sampling points within a single system. In systems serving 3,300\npersons or less, the Department may permit compositing among different systems provided the 5-sample\nlimit is maintained.\n\n     (e) The frequency of monitoring for asbestos shall be in accordance with paragraph (8) of this section;\nthe frequency of monitoring for antimony, arsenic, barium, beryllium, cadmium, chromium, cyanide,\nfluoride, mercury, nickel, selenium and thallium shall be in accordance with paragraph (9) of this section;\nthe frequency of monitoring for nitrate shall be in accordance with paragraph (10) of this section; and the\nfrequency of monitoring for nitrite shall be in accordance with paragraph (11) of this section.\n\n   (8) The frequency of monitoring conducted to determine compliance with the maximum contaminant\nlevel for asbestos specified in Section B(2) above shall be conducted as follows:\n\n    (a) Each community and non-transient, non-community water system is required to monitor for\nasbestos during the first three-year compliance period of each nine-year compliance cycle beginning in the\ncompliance period starting January 1, 1993.\n\n     (b) If the system believes it is not vulnerable to either asbestos contamination in its source water or\ndue to corrosion of asbestos-cement pipe, or both, it may apply to the Department for a waiver of the\nmonitoring requirement in paragraph (8)(a) of this section. If the Department grants the waiver, the system\nis not required to monitor.\n\n    (c) The Department may grant a waiver based on a consideration of the following factors:\n\n         (i)   Potential asbestos contamination of the water source; and,\n\n         (ii) The use of asbestos-cement pipe for finished water distribution and the corrosive nature of the\nwater.\n\n     (d) A waiver remains in effect until the completion of the three-year compliance period. Systems not\nreceiving a waiver must monitor in accordance with the provisions of paragraph (8)(a) of this section.\n\n     (e) A system vulnerable to asbestos contamination due solely to corrosion of asbestos-cement pipe\nshall take one sample at a tap served by asbestos-cement pipe and under conditions where asbestos\ncontamination is most likely to occur.\n\n    (f) A system vulnerable to asbestos contamination due solely to source water shall monitor in\naccordance with the provision of paragraph (7) of this section.\n\n     (g) A system vulnerable to asbestos contamination due both to its source water supply and corrosion\nof asbestos-cement pipe shall take one sample at a tap served by asbestos-cement pipe and under conditions\nwhere asbestos contamination is most likely to occur.\n\n     (h) A system which exceeds the maximum contaminant levels as determined in paragraph (15) of this\nsection shall monitor quarterly beginning in the next quarter after the violation occurred.\n\n     (i) The Department may decrease the quarterly monitoring requirement to the frequency specified in\nparagraph (8)(a) of this section provided the Department has determined that the system is reliably and\nconsistently below the maximum contaminant level. In no case can the Department make this determination\nunless a groundwater system takes a minimum of two quarterly samples and a surface (or combined\nsurface/ground) water system takes a minimum of four quarterly samples.\n\n     (j) If monitoring data collected after January 1, 1990 are generally consistent with the requirements\nof paragraph (8) of this section, then the Department may allow systems to use that data to satisfy the\nmonitoring requirement for the initial compliance period beginning January 1, 1993.\n\n   (9) The frequency of monitoring conducted to determine compliance with the maximum contaminant\nlevels in Section B(2) above for antimony, arsenic, barium, beryllium, cadmium, chromium, cyanide,\nfluoride, mercury, nickel, selenium, and thallium shall be as follows:\n\n     (a) Groundwater systems shall take one sample at each sampling point during each compliance\nperiod. Surface water systems (or combined surface/ground) shall take one sample annually at each\nsampling point.\n\n     (b) The system may apply to the Department for a waiver from the monitoring frequencies specified\nin paragraph (9)(a) of this section. The Department may grant a public water system a waiver for monitoring\nof cyanide, provided that the Department determines that the system is not vulnerable due to lack of any\nindustrial source of cyanide.\n\n       (c) A condition of the waiver shall require that a system shall take a minimum of one sample while\nthe waiver is effective. The term during which the waiver is effective shall not exceed one compliance cycle\n(i.e., nine years).\n\n      (d) The Department may grant a waiver provided surface water systems have monitored annually for\nat least three years and groundwater systems have conducted a minimum of three rounds of monitoring. (At\nleast one sample shall have been taken since January 1, 1990.) Both surface and groundwater systems shall\ndemonstrate that all previous analytical results were less than the maximum contaminant level. Systems\nthat use a new water source are not eligible for a waiver until three rounds of monitoring from the new\nsource have been completed.\n\n    (e) In determining the appropriate reduced monitoring frequency, the Department shall consider:\n\n       (i)   Reported concentrations from all previous monitoring;\n\n       (ii) The degree of variation in reported concentrations; and\n\n      (iii) Other factors which may affect contaminant concentrations such as changes in groundwater\npumping rates, changes in the system's configuration, changes in the system's operating procedures, or\nchanges in stream flows or characteristics.\n\n     (f) A decision by the Department to grant a waiver shall be made in writing and shall set forth the\nbasis for the determination. The determination may be initiated by the Department or upon an application\nby the public water system. The public water system shall specify the basis for its request. The Department\n\nshall review and, where appropriate, revise its determination of the appropriate monitoring frequency when\nthe system submits new monitoring data or when other data relevant to the system's appropriate monitoring\nfrequency become available.\n\n     (g) Systems which exceed the maximum contaminant levels as calculated in paragraph (15) of this\nsection shall monitor quarterly beginning in the next quarter after the violation occurred.\n\n     (h) The Department may decrease the quarterly monitoring requirement to the frequencies specified\nin paragraphs (9)(a) and (9)(c) of this section provided it has determined that the system is reliably and\nconsistently below the maximum contaminant level. In no case can the Department make this determination\nunless a groundwater system takes a minimum of two quarterly samples and a surface water system takes\na minimum of four quarterly samples.\n\n     (i) All new systems or systems that use a new source of water that begin operation after January 22,\n2004 must demonstrate compliance with the MCL within a period of time specified by the Department. The\nsystem must also comply with the initial sampling frequencies specified by the Department to ensure a\nsystem can demonstrate compliance with the MCL. Routine and increased monitoring frequencies shall be\nconducted in accordance with the requirements in this section.\n\n  (10) All public water systems (community; non-transient, non-community; and transient, non-\ncommunity) shall monitor to determine compliance with the maximum contaminant level for nitrate in\nSection B above.\n\n     (a) Community and non-transient, non-community water systems served by groundwater systems\nshall monitor annually beginning January 1, 1993; systems served by surface water shall monitor quarterly\nbeginning January 1, 1993.\n\n     (b) For community and non-transient, non-community water systems, the repeat monitoring\nfrequency for ground water systems shall be quarterly for at least one year following any one sample in\nwhich the concentration is 50 percent or more of the MCL. The Department may allow a groundwater\nsystem to reduce the sampling frequency to annually after four consecutive quarterly samples are reliably\nand consistently less than the MCL.\n\n     (c) For community and non-transient, non-community water systems, the Department may allow a\nsurface water system to reduce the sampling frequency to annually if all analytical results from four\nconsecutive quarters are less than 50 percent of the MCL. A surface water system shall return to quarterly\nmonitoring if any one sample is 50 percent or more of the MCL.\n\n    (d) Each transient non-community water system shall monitor annually beginning January 1, 1993.\n\n    (e) After the initial round of quarterly sampling is completed, each community and non-transient\nnon-community system which is monitoring annually shall take subsequent samples during the quarter(s)\nwhich previously resulted in the highest analytical result.\n\n   (11) All public water systems (community; non-transient, non-community; and transient, non-\ncommunity systems) shall monitor to determine compliance with the maximum contaminant level for nitrite\nin Section B above.\n\n       (v) If a sample result is less than the detection limit, zero will be used to calculate the annual\naverage.\n\n     (l)         [Reserved]\n\n     (m) Analysis for PCBs shall be conducted using EPA-approved methods listed in 40 CFR 141.\n\n           (i)     [Reserved]\n\n           (ii) [Reserved]\n\n       (iii) Compliance with the PCB MCL shall be determined based upon the quantitative results of\nanalyses using EPA-approved methods listed in 40 CFR 141.\n\n      (n) If monitoring data collected after January 1, 1990, are generally consistent with the requirements\nof this section, then the Department may allow systems to use that data to satisfy the monitoring requirement\nfor the initial compliance period beginning January 1, 1993.\n\n     (o) The Department may increase the required monitoring frequency, where necessary, to detect\nvariations within the system (e.g., fluctuations in concentration due to seasonal use, changes in water\nsource).\n\n    (p) The Department has the authority to determine compliance or initiate enforcement action based\nupon analytical results and other information compiled by the Department.\n\n   (q) Each public water system shall monitor at the time designated by the Department within each\ncompliance period.\n\n    (r) Detection as used in this paragraph shall be defined as greater than or equal to the following\nconcentrations for each contaminant.\n\n                 Contaminant                             Detection/Limit\n\n                 Alachlor                                0.0002 mg/L\n\n                 Atrazine                                0.0001 mg/L\n\n                 Benzo[a]pyrene                          0.00002 mg/L\n         \n                 Carbofuran                              0.0009 mg/L\n         \n                 Chlordane                               0.0002 mg/L\n         \n                 Dalapon                                 0.001 mg/L\n         \n                 Dibromochloropropane (DBCP)             0.00002 mg/L\n         \n                 Di (2-ethylhexly) adipate               0.0006 mg/L\n         \n                 Di (2-ethylhexly) phthalate             0.0006 mg/L\n         \n                 Dinoseb                                 0.0002 mg/L\n         \n                 Diquat                                  0.0004 mg/L\n         \n                 2,4-D                                   0.0001 mg/L\n         \n                 Endothall                               0.009 mg/L\n         \n\n                 Endrin                                  0.00001 mg/L\n         \n                 Ethylene dibromide (EDB)                0.00001 mg/L\n         \n         Glyphosate                                      0.006 mg/L\n         \n         Heptachlor                                      0.00004 mg/L\n         \n         Heptachlor epoxide                              0.00002 mg/L\n         \n         Hexachlorobenzene                               0.0001 mg/L\n         \n         Hexachlorocyclopentadiene                       0.0001 mg/L\n         \n         Lindane                                         0.00002 mg/L\n         \n         Methoxychlor                                    0.0001 mg/L\n         \n         Oxamyl                                          0.002 mg/L\n         \n         Picloram                                        0.0001 mg/L\n         \n         Polychlorinated biphenyls (PCBs)\n              (as decachlorobiphenyl)                    0.0001 mg/L\n         \n         Pentachlorophenol                               0.00004 mg/\n         \n         Simazine                                        0.00007 mg/L\n         \n         Toxaphene                                       0.001 mg/L\n         \n         2,3,7,8-TCDD (Dioxin)                           0.000000005 mg/L\n\n         2,4,5-TP (Silvex)                               0.0002 mg/L\n\n     (s) All new systems or systems that used a new source of water that begin operation after January\n22, 2004 must demonstrate compliance with the MCL within a period of time specified by the Department.\nThe system must also comply with the initial sampling frequencies specified by the Department to ensure\na system can demonstrate compliance with the MCL. Routine and increased monitoring frequencies shall\nbe conducted in accordance with the requirements in this section.\n\nF. Maximum Contaminant Levels (MCLs) for Microbiological Contaminants.\n\n  These maximum contaminant levels shall apply to all public water systems.\n\n   (1) Until March 31, 2016, the total coliform MCL is based on the presence or absence of total coliforms\nin a sample, rather than coliform density.\n\n     (a) For a system which collects at least forty (40) samples per month, if no more than five (5.0)\npercent of the samples collected during a month are total coliform-positive, the system is in compliance\nwith the MCL for total coliforms.\n\n      (b) For a system which collects fewer than forty (40) samples per month, if no more than one (1)\nsample collected during a month is total coliform -positive, the system is in compliance with the MCL for\ntotal coliforms.\n\n   (2) Until March 31, 2016, any fecal coliform-positive repeat sample or E. coli-positive repeat sample,\nor any total coliform-positive repeat sample following a fecal coliform-positive or E. coli-positive routine\nsample constitutes a violation of the MCL for total coliforms. For purposes of the public notification\nrequirements in R.61 58.6.E, this is a violation that may pose an acute risk to health.\n\n  (3) Beginning April 1, 2016, a system is in compliance with the MCL for E. coli for samples taken\nunder provisions of R.61-58.17 unless any of the conditions identified in R.61-58.5.F(3)(a) through (d)\noccur. For purposes of the public notification requirements in R.61-58.6.E, violation of the MCL may pose\nan acute risk to health.\n\n"
lines = input.split("\n")
print(len(lines))
preprocessed_input = preprocessing(lines)
print(preprocessed_input)
# paragraph = "Drinking Water : Arsenic       0.010"
predict(preprocessed_input)